In [1]:
import os 
import random
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import GroupShuffleSplit

import tensorflow as tf
from tensorflow import keras, convert_to_tensor, string
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32
from tensorflow import linalg, ones, maximum, newaxis
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Layer, Embedding, MaxPooling1D
from tensorflow.keras.layers import LayerNormalization, ReLU, Dropout
from tensorflow.keras.layers import Activation, Flatten, Conv1D, BatchNormalization
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from keras import backend as K
from keras.backend import softmax

from focal_loss import BinaryFocalLoss
from functions import LRScheduler, TransformerModel 

%load_ext autoreload

%matplotlib inline

### Parameters for plotting model results ###
pd.set_option("display.max_colwidth",100)
sns.set(style="ticks", color_codes=True)
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelpad'] = 5
plt.rcParams['axes.linewidth']= 2
plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'

In [2]:
#! pip install focal-loss

In [3]:
print('tensorflow version: ' + tf. __version__)
print(tf.config.list_physical_devices('CPU'))
print(tf.config.list_physical_devices('GPU'))

tensorflow version: 2.3.4
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# basic random seed
DEFAULT_RANDOM_SEED = 2022

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# tensorflow random seed 
def seedTF(seed=DEFAULT_RANDOM_SEED):
    tf.random.set_seed(seed)
    
# torch random seed
# import torch
# def seedTorch(seed=DEFAULT_RANDOM_SEED):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTF(seed)
    # seedTorch(seed)

seedEverything()

In [5]:
model_name='model2'
#padding_method = 'encode_padding'
#channel = 1
padding_method = 'one_hot_encode_padding'
channel = 4
max_input_length = 600
dropout_rate = 0.1
feature_pad_end= '3end'

import models
custom_model = getattr(models, model_name)
import functions
custom_padding = getattr(functions, padding_method)

In [ ]:
for fold in range(1,6):
    print(fold)
    data_all=pd.read_csv('data/fold'+str(fold)+'.csv')
    data_train=data_all[data_all['set']=='train']
    data_valid=data_all[data_all['set']=='valid']
    
    data_train = data_train.sample(frac=1).reset_index(drop=True)
    data_valid = data_valid.sample(frac=1).reset_index(drop=True)
    
    X_train=custom_padding(data_train, col='seq', seq_len=max_input_length, padding=feature_pad_end, channel=channel)
    y_train=data_train['label']
    X_valid=custom_padding(data_valid, col='seq', seq_len=max_input_length, padding=feature_pad_end, channel=channel)
    y_valid=data_valid['label']
    
    tf.keras.backend.clear_session()

    model=custom_model(seq_length=max_input_length, dropout_rate=dropout_rate)

    #adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
    adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

    model.summary()

    es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
              batch_size=64, epochs=10, verbose=1, callbacks=[es], class_weight={0: 10, 1: 1})

    model.save('model/result/'+model_name+'_'+str(fold))
    with open('model/result/'+model_name+'_'+str(fold)+'.json', 'w') as fp:
        json.dump(model.history.history, fp)

1
5010/5010 [==============================] - 300s 60ms/step - loss: 0.0428 - tp: 150419.0000 - fp: 484.0000 - tn: 158422.0000 - fn: 11271.0000 - accuracy: 0.9633 - precision: 0.9968 - recall: 0.9303 - auc: 0.9979 - prc: 0.9980 - val_loss: 0.0271 - val_tp: 10.0000 - val_fp: 57.0000 - val_tn: 7613.0000 - val_fn: 56.0000 - val_accuracy: 0.9854 - val_precision: 0.1493 - val_recall: 0.1515 - val_auc: 0.6886 - val_prc: 0.0762
Epoch 2/10
5010/5010 [==============================] - 298s 60ms/step - loss: 0.0050 - tp: 160708.0000 - fp: 91.0000 - tn: 158815.0000 - fn: 982.0000 - accuracy: 0.9967 - precision: 0.9994 - recall: 0.9939 - auc: 1.0000 - prc: 1.0000 - val_loss: 0.0304 - val_tp: 8.0000 - val_fp: 7.0000 - val_tn: 7663.0000 - val_fn: 58.0000 - val_accuracy: 0.9916 - val_precision: 0.5333 - val_recall: 0.1212 - val_auc: 0.5819 - val_prc: 0.1673
Epoch 3/10
5010/5010 [==============================] - 298s 59ms/step - loss: 0.0032 - tp: 161118.0000 - fp: 56.0000 - tn: 158850.0000 - fn: 57

In [ ]:
# embedding + cnn

enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

enc_seq_length = 600  # Maximum length of the input sequence
dec_seq_length = enc_seq_length  # Maximum length of the target sequence

h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 32  # Dimensionality of the inner fully connected layer
d_model = 16  # Dimensionality of the model sub-layers' outputs
n = 1  # Number of layers in the encoder stack
 
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers

for fold in range(1,6):
    print(fold)
    data_all=pd.read_csv('data/fold'+str(fold)+'.csv')
    data_train=data_all[data_all['set']=='train']
    data_valid=data_all[data_all['set']=='valid']
    
    data_train = data_train.sample(frac=1).reset_index(drop=True)
    data_valid = data_valid.sample(frac=1).reset_index(drop=True)
    
    X_train=encode_padding(data_train, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_train=data_train['label']
    X_valid=encode_padding(data_valid, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_valid=data_valid['label']
    
    tf.keras.backend.clear_session()

    word_embedding_layer = Embedding(input_dim=enc_vocab_size, output_dim=d_model)
    #training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
    #                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

    inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
    outputs = word_embedding_layer(inputs)
    outputs = Conv1D(activation="relu", input_shape=(enc_seq_length, d_model), filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(32)(outputs)
    outputs = Activation('sigmoid')(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Dense(1)(outputs)
    outputs = Activation('sigmoid')(outputs)
    model = keras.Model(inputs=inputs, outputs=outputs)

    #adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
    adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

    model.summary()

    es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
              batch_size=256, epochs=10, verbose=1, callbacks=[es])

    model.save('model/embedding16_cnn_fold'+str(fold)+'.h5')
    with open('model/embedding16_cnn_fold'+str(fold)+'.json', 'w') as fp:
        json.dump(model.history.history, fp)

In [ ]:
# transformer + cnn

enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

enc_seq_length = 600  # Maximum length of the input sequence
dec_seq_length = enc_seq_length  # Maximum length of the target sequence

h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 32  # Dimensionality of the inner fully connected layer
d_model = 16  # Dimensionality of the model sub-layers' outputs
n = 1  # Number of layers in the encoder stack
 
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers

for fold in range(1,6):
    print(fold)
    data_all=pd.read_csv('data/fold'+str(fold)+'.csv')
    data_train=data_all[data_all['set']=='train']
    data_valid=data_all[data_all['set']=='valid']
    
    data_train = data_train.sample(frac=1).reset_index(drop=True)
    data_valid = data_valid.sample(frac=1).reset_index(drop=True)
    
    X_train=encode_padding(data_train, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_train=data_train['label']
    X_valid=encode_padding(data_valid, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_valid=data_valid['label']
    
    tf.keras.backend.clear_session()

    word_embedding_layer = Embedding(input_dim=enc_vocab_size, output_dim=d_model)
    training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

    inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
    outputs = training_model(inputs, training=True)
    outputs = K.max(outputs,axis=-1)
    outputs = Flatten()(outputs)
    outputs = Dense(32)(outputs)
    outputs = Activation('sigmoid')(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Dense(1)(outputs)
    outputs = Activation('sigmoid')(outputs)
    model = keras.Model(inputs=inputs, outputs=outputs)

    #adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
    adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

    model.summary()

    es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
              batch_size=64, epochs=10, verbose=1, callbacks=[es])

    model.save('model/transformer16_fold'+str(fold)+'.h5')
    with open('model/transformer16_fold'+str(fold)+'.json', 'w') as fp:
        json.dump(model.history.history, fp)

In [ ]:
tf.keras.models.save_model(model, 'model/transformer16_fold'+str(fold)+'.h5')

In [ ]:
# transformer + cnn

enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

enc_seq_length = 600  # Maximum length of the input sequence
dec_seq_length = enc_seq_length  # Maximum length of the target sequence

h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 32  # Dimensionality of the inner fully connected layer
d_model = 16  # Dimensionality of the model sub-layers' outputs
n = 1  # Number of layers in the encoder stack
 
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers

for fold in range(1,6):
    print(fold)
    data_all=pd.read_csv('data/fold'+str(fold)+'.csv')
    data_train=data_all[data_all['set']=='train']
    data_valid=data_all[data_all['set']=='valid']
    
    data_train = data_train.sample(frac=1).reset_index(drop=True)
    data_valid = data_valid.sample(frac=1).reset_index(drop=True)
    
    X_train=encode_padding(data_train, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_train=data_train['label']
    X_valid=encode_padding(data_valid, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=1)
    y_valid=data_valid['label']
    
    tf.keras.backend.clear_session()

    word_embedding_layer = Embedding(input_dim=enc_vocab_size, output_dim=d_model)
    training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

    inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
    outputs = training_model(inputs, training=True)
    #outputs = K.max(outputs,axis=-1)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
    outputs = MaxPooling1D()(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(32)(outputs)
    outputs = Activation('sigmoid')(outputs)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = Dense(1)(outputs)
    outputs = Activation('sigmoid')(outputs)
    model = keras.Model(inputs=inputs, outputs=outputs)

    #adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
    adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

    model.summary()

    es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
              batch_size=64, epochs=10, verbose=1, callbacks=[es])

    model.save('model/transformer16_cnn_fold'+str(fold)+'.h5')
    with open('model/transformer16_cnn_fold'+str(fold)+'.json', 'w') as fp:
        json.dump(model.history.history, fp)

In [ ]:
fold

In [ ]:
enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

enc_seq_length = 600  # Maximum length of the input sequence
dec_seq_length = enc_seq_length  # Maximum length of the target sequence

fold=1
data_all=pd.read_csv('data/fold'+str(fold)+'.csv')
data_train=data_all[data_all['set']=='train']
data_valid=data_all[data_all['set']=='valid']

data_train = data_train.sample(frac=1).reset_index(drop=True)
data_valid = data_valid.sample(frac=1).reset_index(drop=True)

X_train=encode_padding(data_train, col='seq', seq_len=enc_seq_length, padding=CFG.feature_pad_end, channel=1)
y_train=data_train['label']
X_valid=encode_padding(data_valid, col='seq', seq_len=enc_seq_length, padding=CFG.feature_pad_end, channel=1)
y_valid=data_valid['label']


In [ ]:
from functions import LRScheduler, TransformerModel, encode_padding

enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

h = 4  # Number of self-attention heads
d_k = 32  # Dimensionality of the linearly projected queries and keys
d_v = 32  # Dimensionality of the linearly projected values
d_ff = 32  # Dimensionality of the inner fully connected layer
d_model = 128  # Dimensionality of the model sub-layers' outputs
n = 3  # Number of layers in the encoder stack
dropout_rate = 0.1

training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
outputs = training_model(inputs, training=True)
#outputs = K.max(outputs,axis=-1)
outputs = Conv1D(activation="relu", input_shape=(enc_seq_length, d_model), filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Flatten()(outputs)
outputs = Dense(32)(outputs)
outputs = Activation('sigmoid')(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Dense(1)(outputs)
outputs = Activation('sigmoid')(outputs)
model = keras.Model(inputs=inputs, outputs=outputs)

adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

model.summary()

In [ ]:
es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
              batch_size=64, epochs=10, verbose=1, callbacks=[es])

In [ ]:
model.history.history

In [ ]:
from functions import LRScheduler, TransformerModel, encode_padding

enc_vocab_size = 5 # Vocabulary size for the encoder
dec_vocab_size = enc_vocab_size # Vocabulary size for the decoder

h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 32  # Dimensionality of the inner fully connected layer
d_model = 64  # Dimensionality of the model sub-layers' outputs
n = 1  # Number of layers in the encoder stack
dropout_rate = 0.1

word_embedding_layer = Embedding(input_dim=enc_vocab_size, output_dim=d_model)
training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
#outputs = training_model(inputs, training=True)
#outputs = K.max(outputs,axis=-1)
outputs = word_embedding_layer(inputs)
outputs = Conv1D(activation="relu", input_shape=(enc_seq_length, d_model), filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = Conv1D(activation="relu", filters=128, kernel_size=8)(outputs)
outputs = MaxPooling1D()(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Flatten()(outputs)
outputs = Dense(32)(outputs)
outputs = Activation('sigmoid')(outputs)
outputs = Dropout(0.1)(outputs)
outputs = Dense(1)(outputs)
outputs = Activation('sigmoid')(outputs)
model = keras.Model(inputs=inputs, outputs=outputs)

adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

model.summary()

In [ ]:
training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length,
                                      h, d_k, d_v, d_model, d_ff, n, dropout_rate)

inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
outputs = training_model(inputs, training=True)
outputs = K.max(outputs,axis=-1)
outputs = Flatten()(outputs)
outputs = Dense(1)(outputs)
outputs = Activation('sigmoid')(outputs)
model = keras.Model(inputs=inputs, outputs=outputs)

adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 

METRICS = [
          keras.metrics.TruePositives(thresholds=0.5, name='tp'),
          keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
          keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
          keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)
    
model.summary()

In [ ]:
valid_split = GroupShuffleSplit(test_size=.20, n_splits=2)

while True:
    split = valid_split.split(data, y, groups=g)
    train_inds, valid_inds = next(split)
    if len(train_inds)/len(valid_inds)>3:
        break
    
data_train=data.iloc[train_inds,:]
data_valid=data.iloc[valid_inds,:]

In [ ]:
data_valid.shape

In [ ]:
data_train.shape

In [ ]:
data_train_pos=data_train[data_train.label==1]
data_train_pos_new = data_train_pos.sample(n=data_train.shape[0], random_state=1, replace=True)

for i in range(data_train_pos_new.shape[0]):
    r=random.uniform(0, 1)
    if r>0.75:
        tmp=random.sample([-1, -2, -3], k=1)
        data_train_pos_new.seq.iloc[i]=data_train_pos_new.seq.iloc[i][:tmp[0]]
    if r >0.5 and r <=0.75:
        tmp=random.sample([1, 2, 3], k=1)
        data_train_pos_new.seq.iloc[i]=data_train_pos_new.seq.iloc[i][tmp[0]:]
    if r >0.25 and r <=0.5:
        tmp=''.join(np.random.choice(add_on, size=random.sample([1,2,3],k=1), replace=True))
        data_train_pos_new.seq.iloc[i]=data_train_pos_new.seq.iloc[i] + tmp
    else:
        tmp=''.join(np.random.choice(add_on, size=random.sample([1,2,3],k=1), replace=True))
        data_train_pos_new.seq.iloc[i]=tmp + data_train_pos_new.seq.iloc[i]

data_train_all = pd.concat([data_train, data_train_pos_new])

In [ ]:
data_train_all = data_train_all.sample(frac=1).reset_index(drop=True)

In [ ]:
data_train_all['set']='train'
data_valid['set']='valid'

In [ ]:
fold=1

In [ ]:
data_all = pd.concat([data_train_all, data_valid])
data_all.reset_index(drop=True)
data_all['id']=str(fold)+'_'+data_all.index.astype(str)
data_all

In [ ]:
data_all.to_csv('data/fold'+str(fold)+'.csv', index=False)

In [ ]:
X_train=one_hot_encode_padding(data_train_all, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=CFG.channel)
y_train=data_train_all['label']
X_valid=one_hot_encode_padding(data_valid, col='seq', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=CFG.channel)
y_valid=data_valid['label']

In [ ]:
X_valid.shape

In [ ]:
counts = np.bincount(y_valid)
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(y_valid)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]
class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
np.bincount(y_train)

In [ ]:
tf.keras.backend.clear_session()
model = train_model(X_train, y_train, (X_valid, y_valid), channel=CFG.channel,
                   nb_epoch=10, border_mode='same',
                   inp_len=CFG.feature_seq_len, nodes=40, layers=5, nbr_filters=120, filter_len=8, dropout1=0,
                   dropout2=0, dropout3=0.2, patience=3)

In [ ]:
model.history.history

In [ ]:
IRESbase=pd.read_csv('data/IRESbase_new.csv')

In [ ]:
IRESbase=pd.read_csv('data/RF00031.csv')

In [ ]:
IRESbase

In [ ]:
X_test=one_hot_encode_padding(IRESbase, col='IRES.Sequence', seq_len=CFG.feature_seq_len, padding=CFG.feature_pad_end, channel=CFG.channel)

In [ ]:
y_test = model.predict(X_test)

In [ ]:
sum(y_test>0.5)

In [ ]:
y_test = model.predict(X_valid)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
X_valid.shape

In [ ]:
enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence
h = 8  # Number of self-attention heads
d_k = 8  # Dimensionality of the linearly projected queries and keys
d_v = 8  # Dimensionality of the linearly projected values
d_ff = 8  # Dimensionality of the inner fully connected layer
d_model = 8  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack

batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers
 
input_seq = np.random.random((batch_size, input_seq_length))
 
encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))

In [ ]:
input_seq.shape

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
def encode_padding(df, col='utr', seq_len=50, padding='5end', channel=1):
    # 5end padding means pad the left end (5' end) if sequence length < seq_len; keep the seq_len right end (3' end) if  sequence length > seq_len
    # 3end padding means pad the right end (3' end) if sequence length < seq_len; keep the seq_len left end (5' end) if  sequence length > seq_len
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'a':[1],'c':[2],'g':[3],'t':[4], 'n':[0], '(':[5],')':[6],'.':[7]}
    
    # Creat empty matrix.
    vectors=np.zeros([len(df),seq_len,channel])
    
    # Iterate through UTRs and one-hot encode
    for i,seq in enumerate(df[col]):
        if(isinstance(seq, str)):
            if(padding=='3end'):
                seq=seq[:min(len(seq),seq_len)]
            if(padding=='5end'):
                seq=seq[max(0,(len(seq)-seq_len)):len(seq)]
            seq = seq.lower()
            a = np.array([nuc_d[x] for x in seq])
            if(padding=='5end'):
                vectors[i, (seq_len-len(seq)):seq_len] = a
            if(padding=='3end'):
                vectors[i, :len(seq)] = a
    return vectors

In [ ]:
X_train=np.squeeze(encode_padding(data_train_all, col='seq', seq_len=200, padding=CFG.feature_pad_end, channel=1))
y_train=data_train_all['label']
X_valid=np.squeeze(encode_padding(data_valid, col='seq', seq_len=200, padding=CFG.feature_pad_end, channel=1))
y_valid=data_valid['label']
#m_valid=np.squeeze(tf.math.equal(X_valid, 0))*1

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
enc_vocab_size = 64 # Vocabulary size for the encoder
dec_vocab_size = 64 # Vocabulary size for the decoder
 
enc_seq_length = 200  # Maximum length of the input sequence
dec_seq_length = 200  # Maximum length of the target sequence

h = 8  # Number of self-attention heads
d_k = 32  # Dimensionality of the linearly projected queries and keys
d_v = 32  # Dimensionality of the linearly projected values
d_ff = 64  # Dimensionality of the inner fully connected layer
d_model = 64  # Dimensionality of the model sub-layers' outputs
n = 3  # Number of layers in the encoder stack
 
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers

# Create model
training_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)

inputs = tf.keras.layers.Input(shape=(enc_seq_length,))
outputs = training_model(inputs, training=True)
outputs = Flatten()(outputs)
outputs = Dense(1)(outputs)
outputs = Activation('sigmoid')(outputs)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
class LRScheduler(LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, **kwargs):
        super(LRScheduler, self).__init__(**kwargs)
 
        self.d_model = cast(d_model, float32)
        self.warmup_steps = warmup_steps
 
    def __call__(self, step_num):
 
        # Linearly increasing the learning rate for the first warmup_steps, and decreasing it thereafter
        arg1 = step_num ** -0.5
        arg2 = step_num * (self.warmup_steps ** -1.5)
 
        return (self.d_model ** -0.5) * math.minimum(arg1, arg2)

In [ ]:
adam = tf.keras.optimizers.Adam(LRScheduler(d_model), beta_1=0.9, beta_2=0.999, epsilon=1e-08) 

METRICS = [
      keras.metrics.TruePositives(thresholds=0.5, name='tp'),
      keras.metrics.FalsePositives(thresholds=0.5,name='fp'),
      keras.metrics.TrueNegatives(thresholds=0.5,name='tn'),
      keras.metrics.FalseNegatives(thresholds=0.5,name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss=BinaryFocalLoss(gamma=2), metrics=METRICS, optimizer=adam)

es = EarlyStopping(monitor='val_prc', mode='max', verbose=1, patience=3, restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
          batch_size=256, epochs=10, verbose=1, callbacks=[es])

In [ ]:
model.history.history

In [ ]:
training_model.build_graph().summary()

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(IRESbase['IRES.Sequence'].to_numpy(), 
                                              value='N', padding='post', truncating='post', maxlen=600, dtype='object')

In [ ]:
tf.keras.layers.TextVectorization('ATCG')

In [ ]:
vocab_data = ["A", "C", "G", "T"]
max_len = CFG.feature_seq_len

vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len,
 vocabulary=vocab_data)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
t  = Tokenizer(num_words=5,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=False, char_level=True, oov_token=None,
    document_count=0)
fit_text = "ACGTN"
t.fit_on_texts(fit_text)

test_text = "NCGTA"
sequences = t.texts_to_sequences(test_text)

print("sequences : ",sequences,'\n')

print("word_index : ", t.word_index)

In [ ]:
data_valid['seq'].iloc[0]

In [ ]:
test=t.texts_to_sequences(data_valid['seq'].iloc[0])

In [ ]:
class PositionEmbeddingFixedWeights(Layer):
    def __init__(self, sequence_length, vocab_size, output_dim, **kwargs):
        super(PositionEmbeddingFixedWeights, self).__init__(**kwargs)
        word_embedding_matrix = self.get_position_encoding(vocab_size, output_dim)   
        position_embedding_matrix = self.get_position_encoding(sequence_length, output_dim)                                          
        self.word_embedding_layer = Embedding(
            input_dim=vocab_size, output_dim=output_dim,
            weights=[word_embedding_matrix],
            trainable=False
        )
        self.position_embedding_layer = Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix],
            trainable=False
        )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        import numpy as np
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P


    def call(self, inputs):        
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_words = self.word_embedding_layer(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices

In [ ]:
padding_mask

In [ ]:
tf.squeeze(test)

In [ ]:
np.array(test).shape

In [ ]:
len(test)

In [ ]:
IRESbase

In [ ]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json
with open('nucl.json') as f:
    contents = f.readlines()
    t = tokenizer_from_json(contents[0])

In [ ]:
contents[0]